In [43]:
import copy

import pandas as pd
from helper import *
from filenames_and_paths import * 
from typing import *
import multiprocessing as mp
import dask
import joblib

In [ ]:
worker_paths = []
mhw_objects: List[MicrostateHelperWrapper] = []
for i, target_filenames in enumerate(filenames):
    folders.end_folder = paths[i]
    # folders.end_folder = path045
    for filename in target_filenames:
        mhw_objects.append(MicrostateHelperWrapper.static_load(folders, filename + "_th"))
        # worker_paths.append((copy.deepcopy(folders), filename + "_th"))


In [ ]:
# pool = mp.Pool(processes=8)
# mhw_objects: List[MicrostateHelperWrapper] = pool.map(worker_load_mhw, worker_paths)
# pool.close()

In [ ]:
len(mhw_objects)

In [ ]:
def worker(mhw: MicrostateHelperWrapper):
    print("--- Processing: ", mhw.raw_filename)
    
    static_df = nk.microstates_static(mhw.ms_ordered, sampling_rate=2048)
    dynamic_df = nk.microstates_dynamic(mhw.ms_ordered)
    
    static_df["filename"] = mhw.raw_filename
    static_df["subject"] = mhw.folders.end_folder
    
    dynamic_df["filename"] = mhw.raw_filename
    dynamic_df["subject"] = mhw.folders.end_folder
    
    rest_state = mhw.raw_filename.split("_")[2]
    rest_state = rest_state.removeprefix("REST").removeprefix("rs")
    if len(rest_state) == 2:
        rest_state = rest_state[1]
    
    static_df["rest_state"] = rest_state
    dynamic_df["rest_state"] = rest_state
    
    return static_df, dynamic_df

In [ ]:
stat_dyn_arr_joblib = joblib.Parallel(n_jobs=-1, verbose=1)(joblib.delayed(worker)(mhw) for mhw in mhw_objects)

In [ ]:
static_statistics = pd.DataFrame()
dynamic_statistics = pd.DataFrame()

for stat, dyn in stat_dyn_arr_joblib:
    static_statistics = pd.concat([static_statistics, stat])
    dynamic_statistics = pd.concat([dynamic_statistics, dyn])

static_statistics.reset_index(drop=True, inplace=True)
dynamic_statistics.reset_index(drop=True, inplace=True)

In [ ]:
static_statistics.to_csv(folders.save_data + folders.statistics + "static_statistics.csv", index=False)
dynamic_statistics.to_csv(folders.save_data + folders.statistics + "dynamic_statistics.csv", index=False)